In [2]:
import re
import os
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
file_path='results/experiment_a/cc_reno/h1_client.log'
p =10
df_list = [0]*p
df_list = [pd.DataFrame(columns=['time','transfer_rate','bitrate','retransmission', 'cwnd']) for i in range(10)]
with open(file_path, 'r') as f:
    for i, line in enumerate(f):
        idx = i-12
        if idx < 0:
            continue
        if idx % 12 == 10 || idx % 12 == 11:
            continue
        
        # Extract the data from the line
        line = line[6:].strip().split(" ")
        
        # Append the data to the corresponding df
        df_list[(idx)%12] = pd.concat([df_list[(idx)%12], pd.DataFrame([{'time': (idx)//12, 'transfer_rate': float(line[5]), 'bitrate': float(line[8]), 'retransmission': float(line[12]), 'cwnd': float(line[15])}])], ignore_index=True)
        print(line)
        print(df_list[(idx)%12])
        break

In [ ]:
file_path='results/experiment_a/cc_reno/h1_client.log'
p =10
df_list = [0]*p
with open(file_path, 'r') as f:
    for idx, line in enumerate(f):
        if idx <= p+1:
            continue
        
        # Extract the data from the line
        line = line[6:].strip().split(" ")
        
        # Create a list of 10 dfs
        if idx == p+2:
            df_list = [pd.DataFrame(columns=['time','transfer_rate','bitrate','retransmission', 'cwnd']) for i in range(10)]
        
        # Append the data to the corresponding df
        df_list[(idx-2)%p] = pd.concat([df_list[(idx-2)%p], pd.DataFrame([{'time': (idx-2)//p, 'transfer_rate': float(line[5]), 'bitrate': float(line[8]), 'retransmission': float(line[12]), 'cwnd': float(line[15])}])], ignore_index=True)
        print(line)
        print(df_list[(idx-2)%p])
        break

['0.00-1.00', '', '', 'sec', '', '1.25', 'MBytes', '', '10.5', 'Mbits/sec', '', '', '10', '', '', '56.6', 'KBytes']
  time  transfer_rate  bitrate  retransmission  cwnd
0    1           1.25     10.5            10.0  56.6


/tmp/ipykernel_6032/702417606.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_list[(idx-2)%p] = pd.concat([df_list[(idx-2)%p], pd.DataFrame([{'time': (idx-2)//p, 'transfer_rate': float(line[5]), 'bitrate': float(line[8]), 'retransmission': float(line[12]), 'cwnd': float(line[15])}])], ignore_index=True)


In [24]:
df_list[0]

,time,transfer_rate,bitrate,retransmission,cwnd
0,1,1.25,10.5,10.0,56.6


In [ ]:
def parse_iperf_log(file_path):
    """
    Parse iperf3 log file lines that contain per-interval statistics.
    Expected line format (sample):
       [  5]   0.00-1.00   sec  1.88 MBytes  15.7 Mbits/sec    0    925 KBytes       
    """
    # Regex to capture: start-end time, transfer (MBytes), throughput (Mbits/sec), retransmissions, cwnd and its unit.
    pattern = re.compile(
        r'\[\s*\d+\]\s+(\d+\.\d+)-(\d+\.\d+)\s+sec\s+([\d\.]+)\s+MBytes\s+([\d\.]+)\s+Mbits/sec\s+(\d+)\s+.*Cwnd\s+([\d\.]+)\s*(KBytes|MBytes)'
    )
    data = []
    with open(file_path, 'r') as f:
        for line in f:
            
    # return pd.DataFrame(data)


In [12]:
df=parse_iperf_log('results/experiment_a/cc_reno/h1_client.log')
df

""


In [13]:
print(df)

Empty DataFrame
Columns: []
Index: []


In [4]:
def plot_throughput(df, title, output_dir):
    """Plot throughput over time."""
    plt.figure(figsize=(8, 4))
    plt.plot(df['end'], df['throughput_Mbps'], marker='o', linestyle='-')
    plt.xlabel('Time (s)')
    plt.ylabel('Throughput (Mbit/s)')
    plt.title(title)
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, f"{title.replace(' ', '_')}_throughput.png"))
    plt.close()

In [5]:
def plot_cwnd(df, title, output_dir):
    """Plot congestion window (cwnd) over time."""
    plt.figure(figsize=(8, 4))
    plt.plot(df['end'], df['cwnd_bytes'] / 1024, marker='o', linestyle='-', color='green')
    plt.xlabel('Time (s)')
    plt.ylabel('Cwnd (KBytes)')
    plt.title(title)
    plt.grid(True)
    plt.savefig(os.path.join(output_dir, f"{title.replace(' ', '_')}_cwnd.png"))
    plt.close()


In [6]:
def summarize_metrics(df):
    """Calculate summary metrics."""
    avg_throughput = df['throughput_Mbps'].mean()
    total_retrans = df['retransmissions'].sum()
    max_cwnd = df['cwnd_bytes'].max()
    
    # Goodput: For our purposes we use throughput as a proxy.
    goodput = avg_throughput  # In practice, goodput should discount retransmissions
    
    # Assume a simple model: if we expect no losses then loss rate can be approximated as retransmissions
    # divided by an estimated number of segments (here we use a dummy value since real packet counts
    # require knowing the MSS and total bytes transferred)
    # For example, if each stream transfers 1.88 MB per interval over 10 streams:
    total_segments = len(df) * 10 * 100  # dummy estimate; replace with actual calculation if available
    loss_rate = (total_retrans / total_segments) * 100  # in percentage
    
    return avg_throughput, goodput, loss_rate, max_cwnd

In [7]:
def process_log_for_cc(cc_folder, cc_name):
    """
    Process the iperf3 log for a given congestion control algorithm.
    cc_folder: folder containing the log files (e.g., 'partA' or 'partB')
    cc_name: congestion control algorithm name (e.g., 'reno')
    """
    log_file = os.path.join(cc_folder, f"{cc_name}.txt")
    if not os.path.isfile(log_file):
        print(f"Log file {log_file} not found.")
        return
    
    df = parse_iperf_log(log_file)
    
    # Create an output folder for plots
    plots_dir = os.path.join(cc_folder, 'plots')
    os.makedirs(plots_dir, exist_ok=True)
    
    # Generate throughput and cwnd plots
    plot_throughput(df, f"{cc_name.upper()} Throughput", plots_dir)
    plot_cwnd(df, f"{cc_name.upper()} CWND", plots_dir)
    
    # Compute summary metrics
    avg_tp, goodput, loss_rate, max_cwnd = summarize_metrics(df)
    
    summary = {
        'Congestion Control': cc_name.upper(),
        'Average Throughput (Mbit/s)': round(avg_tp, 2),
        'Goodput (Mbit/s)': round(goodput, 2),
        'Packet Loss Rate (%)': round(loss_rate, 4),
        'Max CWND (Bytes)': int(max_cwnd)
    }
    
    return summary, df

In [10]:
# Define the folders where logs are stored (change as needed)
cc_list = ['reno', 'vegas', 'htcp']
# For example, we analyze logs from partA folder:
folder = 'results/experiment_a/cc_reno'

In [ ]:
summaries = []
for cc in cc_list:
    summary, df = process_log_for_cc(folder, cc)
    if summary:
        summaries.append(summary)
        print(f"Summary for {cc.upper()}:")
        print(summary)
        print("-" * 40)

# Create a summary table
summary_df = pd.DataFrame(summaries)
summary_csv = os.path.join(folder, "summary_metrics.csv")
summary_df.to_csv(summary_csv, index=False)
print(f"Summary metrics saved to {summary_csv}")